# Experimenting with Depth First Search and Best First Search algorithms.

### The Graph
There are two popular options for representing a graph, 
- adjacency matrix (effective with dense graphs), and 
- adjacency list (effective with sparse graphs) 

Here the graph is implemented using an adjacency list which stores each node in a dictionary along with a set containing their adjacent nodes. 

As the graph is undirected each edge is stored in both incident nodes adjacent sets.

In [1]:
graph = {'A': set(['B', 'C']),
         'B': set(['A', 'D', 'E']),
         'C': set(['A', 'F']),
         'D': set(['B']),
         'E': set(['B', 'F']),
         'F': set(['C', 'E'])}
print(graph)

{'A': {'B', 'C'}, 'B': {'D', 'A', 'E'}, 'C': {'F', 'A'}, 'D': {'B'}, 'E': {'B', 'F'}, 'F': {'E', 'C'}}


<img src="fig1.png" width=170 height=170 />

### Depth-First Search

Depth-First search explores possible vertices (from a supplied root) down each branch before backtracking. 

The algorithm works as follows:

- Mark the current vertex as being visited.
- Explore each adjacent vertex that is not included in the visited set.

### Connected Component
The implementation below uses the stack data-structure to build-up and return a set of vertices that are accessible within the subjects connected component. Using Python’s overloading of the subtraction operator to remove items from a set, we are able to add only the unvisited adjacent vertices.

In [2]:
def dfs(graph, start):
    visited, stack = [], [start] # lists
    while stack:
        vertex = stack.pop()
        if vertex not in visited:
            visited.append(vertex)
            stack.extend(graph[vertex] - set(visited)) 
    return visited

dfs(graph, 'A') # ['A', 'B', 'E', 'F', 'C', 'D'] # Correct output

['A', 'C', 'F', 'E', 'B', 'D']

The second implementation provides the same functionality as the first, however, this time we are using the more succinct recursive form. Due to a common Python gotcha with default parameter values being created only once, we are required to create a new visited set on each user invocation. Another Python language detail is that function variables are passed by reference, resulting in the visited mutable set not having to reassigned upon each recursive call.

In [ ]:
def dfs(graph, start, visited=[]): 
    visited.append(start)
    for next in graph[start] - set(visited): 
        dfs(graph, next, visited)
    return visited   # returning all the visited nodes

dfs(graph, 'A') 

### Paths
We are able to tweak both of the previous implementations to return all possible paths between a start and goal vertex. The implementation below uses the stack data-structure again to iteratively solve the problem, yielding each possible path when we locate the goal. Using a generator allows the user to only compute the desired amount of alternative paths.

In [ ]:
def dfs_paths(graph, start, goal, path=None):
    if path is None:
        path = [start]
    if start == goal:
        yield path
    for next in graph[start] - set(path):
        yield from dfs_paths(graph, next, goal, path + [next])

list(dfs_paths(graph, 'A', 'F')) 

### Breath-First Search
An alternative algorithm called Breath-First search provides us with the ability to return the same results as DFS but with the added guarantee to return the shortest-path first. This algorithm is a little more tricky to implement in a recursive manner instead using the queue data-structure, as such I will only being documenting the iterative approach. The actions performed per each explored vertex are the same as the depth-first implementation, however, replacing the stack with a queue will instead explore the breadth of a vertex depth before moving on. This behavior guarantees that the first path located is one of the shortest-paths present, based on number of edges being the cost factor.

Connected Component Similar to the iterative DFS implementation the only alteration required is to remove the next item from the beginning of the list structure instead of the stacks last.

In [ ]:
def bfs(graph, start):
    visited, queue = [], [start] # Use list
    while queue:
        vertex = queue.pop(0)
        if vertex not in visited:
            visited.append(vertex)
            queue.extend(graph[vertex] - set(visited)) # Convert list to set()
    return visited

bfs(graph, 'A') 

### Paths
This implementation can again be altered slightly to instead return all possible paths between two vertices, the first of which being one of the shortest such path.

In [ ]:
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for next in graph[vertex] - set(path):
            if next == goal:
                yield path + [next]
            else:
                queue.append((next, path + [next]))

list(bfs_paths(graph, 'A', 'F')) 

Knowing that the shortest path will be returned first from the BFS path generator method we can create a useful method which simply returns the shortest path found or ‘None’ if no path exists. As we are using a generator this in theory should provide similar performance results as just breaking out and returning the first matching path in the BFS implementation.

In [ ]:
def shortest_path(graph, start, goal):
    try:
        return next(bfs_paths(graph, start, goal))
    except StopIteration:
        return None

shortest_path(graph, 'A', 'F') 